In [1]:
import petl as pt
import pypyodbc as bd

In [2]:
#extract Data from CSV

In [3]:
personne = pt.fromcsv('Personne.csv')

In [4]:
medecin = pt.fromcsv('Medecin.csv')

In [5]:
consultation = pt.fromcsv('CONSULTATION.csv')

In [7]:
#Transformation data

In [8]:
#Dimension Personne
personne = pt.setheader(personne,['id_pers','nom','pernom','tel','adresse','sexe'])

In [9]:
personne = pt.cutout(personne,'pernom') #select all columns without 'prenom'

In [10]:
personne

id_pers,nom,tel,adresse,sexe
811,Matt,1911244512,Adresse5,F
711,Ann,942228240,Adresse5,F
911,Laila,8019547231,Adresse4,M
688,Matt,5974928000,Adresse4,F
47,Ann,8229280313,Adresse1,F


In [11]:
#Dimension Medicien
medecin = medecin.rename('ID','id_med')

In [12]:
medecin

id_med,tel,adresse,specialite
4,2322242318,Addresse 202,Neurologie
31,6219616626,Addresse 302,La dermatologie
20,2252827932,Addresse 822,L'andrologie
40,3794017827,Addresse 100,Stomatologie
199,6150399869,Addresse 100,L'andrologie


In [13]:
#Join Data :
consultation = pt.join(medecin,consultation,key='id_med')

In [14]:
consultation = pt.join(personne,consultation,key='id_pers')

In [15]:
#moving position of columns
consultation = pt.movefield(consultation,'id_med',0)

In [16]:
#select the important columns
consultation = pt.cut(consultation,'id_med','id_pers')

In [17]:
#aggregation   
consultation = pt.aggregate(consultation,['id_med','id_pers'],len).rename('value','NbrConsultation')

In [18]:
#result
consultation.lookall()

+--------+---------+-----------------+
| id_med | id_pers | NbrConsultation |
+========+=========+=================+
| '12'   | '513'   |               1 |
+--------+---------+-----------------+
| '12'   | '711'   |               1 |
+--------+---------+-----------------+
| '12'   | '811'   |               1 |
+--------+---------+-----------------+
| '20'   | '711'   |               1 |
+--------+---------+-----------------+
| '20'   | '830'   |               2 |
+--------+---------+-----------------+
| '23'   | '175'   |               1 |
+--------+---------+-----------------+
| '23'   | '47'    |               1 |
+--------+---------+-----------------+
| '28'   | '175'   |               3 |
+--------+---------+-----------------+
| '29'   | '513'   |               1 |
+--------+---------+-----------------+
| '29'   | '711'   |               1 |
+--------+---------+-----------------+
| '29'   | '830'   |               1 |
+--------+---------+-----------------+
| '29'   | '911'   |     

In [19]:
#same stats
print(consultation['NbrConsultation'].max())
print(consultation['NbrConsultation'].min())

5
1


In [22]:
consultation['NbrConsultation']

AttributeError: 'ValuesView' object has no attribute 'sorted'

In [ ]:
#rename id of personne & medecin for handling database errors

In [ ]:
personne = pt.rename(personne,'id_pers','id')

In [ ]:
personne

In [ ]:
medecin = pt.rename(medecin,'id_med','id')

In [ ]:
medecin 

In [ ]:
#--------------------------------------------Load Data--------------------------------------------#

In [ ]:
#connect with sql server
cn = bd.connect(Trusted_Connection='yes',driver='{SQL Server}',server='DESKTOP-LORG5OV',database='Data_Mart_hopital')

In [ ]:
#load data to datawarehouse
pt.todb(personne,cn,'Dim_PERSONNE')

In [ ]:
pt.todb(medecin,cn,'Dim_MEDECIN')

In [ ]:
pt.todb(consultation,cn,'Fait_CONSULTATION')